In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import time

In [ ]:
# Set up headless Chrome
options = Options()
#options.add_argument("--headless=new")  # Use "--headless" for older Chrome
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

In [ ]:
url = "https://www.uniqlo.com/de/de/products/E478984-000/00?colorDisplayCode=00&sizeDisplayCode=004"

In [ ]:
driver = webdriver.Chrome(options=options)


# Retryable size extraction
def get_size_info_with_retry(driver, index, max_retries=3):
    for attempt in range(max_retries):
        try:
            # Refetch fresh wrapper every time
            wrappers = driver.find_elements(By.CSS_SELECTOR, 'div.size-chip-wrapper')
            wrapper = wrappers[index]

            button = wrapper.find_element(By.CSS_SELECTOR, 'button[data-testid="ITOChip"]')
            label_div = button.find_element(By.CSS_SELECTOR, '[data-testid="ITOTypography"]')
            size_label = driver.execute_script("return arguments[0].textContent;", label_div).strip()

            # Availability
            has_strike = len(wrapper.find_elements(By.CSS_SELECTOR, 'div.strike')) > 0
            available = not has_strike

            return size_label, available

        except StaleElementReferenceException as e:
            print(f"Retry {attempt + 1} due to stale element")
            time.sleep(0.5)

    return "N/A", False

sizes_dict = {}  # Dictionary to store size -> availability

try:
    driver.get(url)

    # Wait for size elements to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.size-chip-wrapper'))
    )

    # List all sizes
    # Get the number of size wrappers (do this only once!)
    total = len(driver.find_elements(By.CSS_SELECTOR, 'div.size-chip-wrapper'))

    for i in range(total):
        size_label, available = get_size_info_with_retry(driver, i)
        sizes_dict[size_label] = available  # Store in dictionary
    

    # Print the results
    for size, available in sizes_dict.items():
        print(f"Size: {size} | Available: {available}")
        
    # 🔍 Find the selected size
    selected_wrapper = driver.find_element(By.CSS_SELECTOR, 'div.size-chip-wrapper.size-chip-selected')
    selected_label = selected_wrapper.find_element(By.CSS_SELECTOR, '[data-testid="ITOTypography"]')
    selected_size = driver.execute_script("return arguments[0].textContent;", selected_label).strip()

    print(f"\n✅ Selected Size: {selected_size}")

    # Find product name
    # Wait for the <h1> element to be present
    product_name = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "main > div > div > h1"))
    )

    # Extract and print the text
    product_name = product_name.text.strip()
    print(f"\n Product name: {product_name}")
finally:
    driver.quit()
    print("done")

In [ ]:
# TODO: next step: add mail notification system

# send mail with template

"""
Important notes:

You need to use an "App Password" instead of your regular Gmail password. 
Create an app password here: https://myaccount.google.com/apppasswords
This requires enabling 2-Step Verification in your Google Account settings.
"""
import smtplib
from email.mime.text import MIMEText
from email.header import Header

# replace all values
sender_email = "" 
sender_app_password = ""
subject = f"Uniqlo: {product_name} available!"

# fill in the text that should be sent to the therapists
body = f"""
Your item became available! {url}
"""

def send_mail(url, recipient_email):
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(sender_email, sender_app_password)
        message = MIMEText(body, _charset="utf-8")
        message['Subject'] = Header(subject, 'utf-8')
        message['From'] = sender_email
        message['To'] = recipient_email
        print(message)
        server.sendmail(sender_email, recipient_email, message.as_string())

        
send_mail(url, sender_email)